https://spell.ml/blog/pytorch-quantization-X8e7wBAAACIAHPhT

https://pytorch.org/tutorials/intermediate/quantized_transfer_learning_tutorial.html

https://zhuanlan-zhihu-com.translate.goog/p/299108528?_x_tr_sl=auto&_x_tr_tl=en&_x_tr_hl=en-US&_x_tr_pto=wapp

https://zhuanlan-zhihu-com.translate.goog/p/149659607?_x_tr_sl=auto&_x_tr_tl=en&_x_tr_hl=en-US&_x_tr_pto=wapp

https://github.com/IntelLabs/distiller/issues/327 -> Quantizing a MNIST

In [1]:
import numpy as np
# import pandas as pd
import torch
import torch.nn as nn
# import torchvision
# from torch.utils.data import DataLoader
# from torchvision import datasets
# import torchvision.transforms as transforms
# import os
# import time
import sys
import torch.quantization
# from loss import TripletLoss
# import matplotlib.pyplot as plt
# from PIL import Image as Img
torch.set_printoptions(precision=30)

/home/avishek/anaconda3/envs/py39/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


#### TEST 1
##### No calibration with Input Images and weight set to +1 or -1 and bias only

In [2]:
class M(nn.Module):

    def __init__(self):
            super(M, self).__init__()
            # QuantStub converts tensors from floating point to quantized
            self.quant = torch.quantization.QuantStub()
            # self.BN = nn.BatchNorm2d(3)
            # self.conv = torch.nn.Conv2d(1, 1, 1)
            # self.conv = torch.nn.Conv2d(in_channels=3, out_channels=2, kernel_size=3, stride=1, padding=1)
            self.conv = torch.nn.Conv2d(in_channels=1, out_channels=1, kernel_size=3, stride=1, padding=1)
            # self.conv.weight = torch.nn.Parameter(torch.tensor([[[[ 0.03307433053851127625, -0.13484150171279907227, -0.21625524759292602539], 
            #                                                         [ 0.14247404038906097412, -0.14247404038906097412, -0.24932956695556640625], 
            #                                                         [ 0.32311078906059265137, -0.14501821994781494141, -0.21371106803417205811]]]]))
            self.conv.weight = torch.nn.Parameter(torch.tensor([[[[ 1.0, 1.0, -1.0], 
                                                                    [ -1.0, 1.0, 1.0], 
                                                                    [ 1.0, 1.0, -1.0]]]]))
            self.conv.bias = torch.nn.Parameter(torch.tensor([10.0]))
            # self.relu = torch.nn.ReLU()
            # DeQuantStub converts tensors from quantized to floating point
            self.dequant = torch.quantization.DeQuantStub()

    def forward(self, x):

        x = self.quant(x)
        # x = self.BN(x)
        x = self.conv(x)
        # x = self.relu(x)      
        x = self.dequant(x)
         
        return x

In [75]:
# model_fp32.conv.weight = torch.nn.Parameter(torch.tensor([[[[ 0.0, 0.0, 0.0],
#           [ 0.0,  1.0, 0.0],
#           [ 0.0, 0.0, 0.0]]]]))
# model_fp32.conv.bias = torch.nn.Parameter(torch.tensor([0.0]))

In [86]:
# model_fp32.conv.weight = torch.nn.Parameter(torch.tensor([[[[ 0.0, 0.0, 0.0],
#           [ 0.0,  1.0, 0.0],
#           [ 0.0, 0.0, 0.0]]]]))
# model_fp32.conv.weight = torch.nn.Parameter(torch.tensor([[[[ 0.03307433053851127625, -0.13484150171279907227, -0.21625524759292602539], 
# [ 0.14247404038906097412, -0.14247404038906097412, -0.24932956695556640625], 
# [ 0.32311078906059265137, -0.14501821994781494141, -0.21371106803417205811]]]]))
# model_fp32.conv.bias = torch.nn.Parameter(torch.tensor([0.1095]))

In [3]:
model_fp32 = M()
model_fp32.eval()
model_fp32.qconfig = torch.quantization.get_default_qconfig('fbgemm')
# model_fp32_fused = torch.quantization.fuse_modules(
#     model_fp32, [['conv', 'relu']])
# model_fp32_fused = torch.quantization.fuse_modules(
#     model_fp32, ['conv'])
model_fp32_prepared = torch.quantization.prepare(model_fp32)
model_fp32_converted = torch.quantization.convert(model_fp32_prepared, inplace=True)

/home/avishek/anaconda3/envs/py39/lib/python3.9/site-packages/torch/ao/quantization/observer.py:172: UserWarning: Please use quant_min and quant_max to specify the range for observers.                     reduce_range will be deprecated in a future release of PyTorch.
  warnings.warn(
/home/avishek/anaconda3/envs/py39/lib/python3.9/site-packages/torch/ao/quantization/observer.py:1107: UserWarning: must run observer before calling calculate_qparams.                                    Returning default scale and zero point 
  warnings.warn(


In [4]:
model_fp32.qconfig

QConfig(activation=functools.partial(<class 'torch.ao.quantization.observer.HistogramObserver'>, reduce_range=True){}, weight=functools.partial(<class 'torch.ao.quantization.observer.PerChannelMinMaxObserver'>, dtype=torch.qint8, qscheme=torch.per_channel_symmetric){})

In [5]:
model_fp32_converted.quant.zero_point

tensor([0])

In [11]:
model_fp32_converted.quant.scale

tensor([1.])

In [7]:
model_fp32_converted.conv._weight_bias()

(tensor([[[[ 0.996078491210937500000000000000,  0.996078491210937500000000000000,
            -0.996078491210937500000000000000],
           [-0.996078491210937500000000000000,  0.996078491210937500000000000000,
             0.996078491210937500000000000000],
           [ 0.996078491210937500000000000000,  0.996078491210937500000000000000,
            -0.996078491210937500000000000000]]]], size=(1, 1, 3, 3),
        dtype=torch.qint8, quantization_scheme=torch.per_channel_affine,
        scale=tensor([0.007843137718737125396728515625], dtype=torch.float64),
        zero_point=tensor([0]), axis=0),
 Parameter containing:
 tensor([10.], requires_grad=True))

In [8]:
model_fp32_converted.conv.zero_point

0

In [9]:
model_fp32_converted.conv.scale

1.0

In [13]:
model_fp32_converted

M(
  (quant): Quantize(scale=tensor([1.]), zero_point=tensor([0]), dtype=torch.quint8)
  (conv): QuantizedConv2d(1, 1, kernel_size=(3, 3), stride=(1, 1), scale=1.0, zero_point=0, padding=(1, 1))
  (dequant): DeQuantize()
)

In [14]:
model_fp32_converted.conv.weight()

tensor([[[[ 0.996078491210937500000000000000,  0.996078491210937500000000000000,
           -0.996078491210937500000000000000],
          [-0.996078491210937500000000000000,  0.996078491210937500000000000000,
            0.996078491210937500000000000000],
          [ 0.996078491210937500000000000000,  0.996078491210937500000000000000,
           -0.996078491210937500000000000000]]]], size=(1, 1, 3, 3),
       dtype=torch.qint8, quantization_scheme=torch.per_channel_affine,
       scale=tensor([0.007843137718737125396728515625], dtype=torch.float64),
       zero_point=tensor([0]), axis=0)

In [54]:
model_fp32_converted.conv._weight_bias()

(tensor([[[[ 0.996078491210937500000000000000,  0.996078491210937500000000000000,
            -0.996078491210937500000000000000],
           [-0.996078491210937500000000000000,  0.996078491210937500000000000000,
             0.996078491210937500000000000000],
           [ 0.996078491210937500000000000000,  0.996078491210937500000000000000,
            -0.996078491210937500000000000000]]]], size=(1, 1, 3, 3),
        dtype=torch.qint8, quantization_scheme=torch.per_channel_affine,
        scale=tensor([0.007843137718737125396728515625], dtype=torch.float64),
        zero_point=tensor([0]), axis=0),
 Parameter containing:
 tensor([110.], requires_grad=True))

In [52]:
model_fp32_converted.conv.bias

<bound method Conv2d.bias of QuantizedConv2d(1, 1, kernel_size=(3, 3), stride=(1, 1), scale=1.0, zero_point=0, padding=(1, 1))>

In [63]:
model_fp32_converted.conv

QuantizedConv2d(1, 1, kernel_size=(3, 3), stride=(1, 1), scale=1.0, zero_point=0, padding=(1, 1))

In [38]:
model_fp32_converted.dequant

DeQuantize()

In [14]:
input_fp32 = torch.tensor([[[[ 0.95466703176498413086, -1.36212718486785888672,
            0.75253891944885253906,  1.57104063034057617188],
          [ 0.97250884771347045898, -0.67004448175430297852,
           -0.58047348260879516602,  1.30683445930480957031],
          [-0.13423979282379150391,  0.16391958296298980713,
           -0.71688455343246459961,  0.05846109613776206970],
          [ 1.07569837570190429688, -0.06351475417613983154,
           -0.19469638168811798096, -0.09430617839097976685]]]], requires_grad=False)

In [16]:
input_fp32.min()

tensor(-1.362127184867858886718750000000)

In [17]:
input_fp32.max()

tensor(1.571040630340576171875000000000)

In [18]:
(input_fp32.max() - input_fp32.min())/255

tensor(0.011502618901431560516357421875)

In [66]:
# quantization algorithm calibration happens here
# this example uses just a single sample, but obvious in prod you will
# want to use some meaningful subset of your training or test set
# instead.
# input_fp32 = torch.randn(2, 3, 4, 4)
# input_fp32 = torch.randn(1, 1, 4, 4)

model_fp32_converted(input_fp32)

# model_int8 = torch.quantization.convert(model_fp32_prepared)
# res = model_int8(input_fp32)

tensor([[[[13., 12., 13., 13.],
          [13., 10., 11., 15.],
          [13., 13., 10., 12.],
          [12., 10., 11., 11.]]]])

##### Test 2
##### Weight set to 1 at center and bias is 0. This will pass in the image as it is. So output will be equal to input

In [9]:
class M1(nn.Module):

    def __init__(self):
            super(M1, self).__init__()
            # QuantStub converts tensors from floating point to quantized
            self.quant = torch.quantization.QuantStub()
            # self.BN = nn.BatchNorm2d(3)
            # self.conv = torch.nn.Conv2d(1, 1, 1)
            # self.conv = torch.nn.Conv2d(in_channels=3, out_channels=2, kernel_size=3, stride=1, padding=1)
            self.conv = torch.nn.Conv2d(in_channels=1, out_channels=1, kernel_size=3, stride=1, padding=1)
            # self.conv.weight = torch.nn.Parameter(torch.tensor([[[[ 0.03307433053851127625, -0.13484150171279907227, -0.21625524759292602539], 
            #                                                         [ 0.14247404038906097412, -0.14247404038906097412, -0.24932956695556640625], 
            #                                                         [ 0.32311078906059265137, -0.14501821994781494141, -0.21371106803417205811]]]]))
            self.conv.weight = torch.nn.Parameter(torch.tensor([[[[ 0., 0., 0.],
                                                                    [0., 1., 0.],
                                                                    [0., 0., 0.]]]]), requires_grad=False)
            self.conv.bias = torch.nn.Parameter(torch.tensor([0.0]))
            # self.relu = torch.nn.ReLU()
            # DeQuantStub converts tensors from quantized to floating point
            self.dequant = torch.quantization.DeQuantStub()

    def forward(self, x):

        x = self.quant(x)
        # x = self.BN(x)
        x = self.conv(x)
        # x = self.relu(x)      
        x = self.dequant(x)
         
        return x

In [10]:
model_fp32_1 = M1()
model_fp32_1.eval()
model_fp32_1.qconfig = torch.quantization.get_default_qconfig('fbgemm')
# model_fp32_fused = torch.quantization.fuse_modules(
#     model_fp32, [['conv', 'relu']])
# model_fp32_fused = torch.quantization.fuse_modules(
#     model_fp32, ['conv'])
model_fp32_prepared_1 = torch.quantization.prepare(model_fp32_1)
model_fp32_prepared_1(input_fp32)   ### Passing the input through model before conversion for calibration
model_fp32_converted_1 = torch.quantization.convert(model_fp32_prepared_1, inplace=True)

/home/avishek/anaconda3/envs/py39/lib/python3.9/site-packages/torch/ao/quantization/observer.py:886: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  src_bin_begin // dst_bin_width, 0, self.dst_nbins - 1
/home/avishek/anaconda3/envs/py39/lib/python3.9/site-packages/torch/ao/quantization/observer.py:891: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode=

In [27]:
model_fp32_1.qconfig

QConfig(activation=functools.partial(<class 'torch.ao.quantization.observer.HistogramObserver'>, reduce_range=True){}, weight=functools.partial(<class 'torch.ao.quantization.observer.PerChannelMinMaxObserver'>, dtype=torch.qint8, qscheme=torch.per_channel_symmetric){})

In [12]:
model_fp32_converted_1.quant

Quantize(scale=tensor([0.019509643316268920898437500000]), zero_point=tensor([49]), dtype=torch.quint8)

In [15]:
input_fp32

tensor([[[[ 0.954667031764984130859375000000, -1.362127184867858886718750000000,
            0.752538919448852539062500000000,  1.571040630340576171875000000000],
          [ 0.972508847713470458984375000000, -0.670044481754302978515625000000,
           -0.580473482608795166015625000000,  1.306834459304809570312500000000],
          [-0.134239792823791503906250000000,  0.163919582962989807128906250000,
           -0.716884553432464599609375000000,  0.058461096137762069702148437500],
          [ 1.075698375701904296875000000000, -0.063514754176139831542968750000,
           -0.194696381688117980957031250000, -0.094306178390979766845703125000]]]])

In [13]:
model_fp32_converted_1.quant(input_fp32)

tensor([[[[ 0.955972552299499511718750000000, -0.955972552299499511718750000000,
            0.760876059532165527343750000000,  1.580281138420104980468750000000],
          [ 0.975482165813446044921875000000, -0.663327872753143310546875000000,
           -0.585289299488067626953125000000,  1.307146072387695312500000000000],
          [-0.136567503213882446289062500000,  0.156077146530151367187500000000,
           -0.721856832504272460937500000000,  0.058528929948806762695312500000],
          [ 1.073030352592468261718750000000, -0.058528929948806762695312500000,
           -0.195096433162689208984375000000, -0.097548216581344604492187500000]]]],
       size=(1, 1, 4, 4), dtype=torch.quint8,
       quantization_scheme=torch.per_tensor_affine, scale=0.01950964331626892,
       zero_point=49)

In [28]:
model_fp32_converted_1.conv

QuantizedConv2d(1, 1, kernel_size=(3, 3), stride=(1, 1), scale=0.01950964331626892, zero_point=49, padding=(1, 1))

In [30]:
model_fp32_converted_1.conv.zero_point

49

In [31]:
model_fp32_converted_1.conv.scale

0.01950964331626892

In [29]:
model_fp32_converted_1(input_fp32)

tensor([[[[ 0.955972552299499511718750000000, -0.955972552299499511718750000000,
            0.760876059532165527343750000000,  1.580281138420104980468750000000],
          [ 0.975482165813446044921875000000, -0.663327872753143310546875000000,
           -0.585289299488067626953125000000,  1.307146072387695312500000000000],
          [-0.136567503213882446289062500000,  0.156077146530151367187500000000,
           -0.721856832504272460937500000000,  0.058528929948806762695312500000],
          [ 1.073030352592468261718750000000, -0.058528929948806762695312500000,
           -0.195096433162689208984375000000, -0.097548216581344604492187500000]]]])

In [34]:
model_fp32_converted_1.conv._weight_bias()

(tensor([[[[0.000000000000000000000000000000, 0.000000000000000000000000000000,
            0.000000000000000000000000000000],
           [0.000000000000000000000000000000, 0.996078491210937500000000000000,
            0.000000000000000000000000000000],
           [0.000000000000000000000000000000, 0.000000000000000000000000000000,
            0.000000000000000000000000000000]]]], size=(1, 1, 3, 3),
        dtype=torch.qint8, quantization_scheme=torch.per_channel_affine,
        scale=tensor([0.007843137718737125396728515625], dtype=torch.float64),
        zero_point=tensor([0]), axis=0),
 Parameter containing:
 tensor([0.], requires_grad=True))

In [39]:
model_fp32_converted_1.conv.bias

<bound method Conv2d.bias of QuantizedConv2d(1, 1, kernel_size=(3, 3), stride=(1, 1), scale=0.01950964331626892, zero_point=49, padding=(1, 1))>

In [105]:
model_fp32_converted.conv.weight()

tensor([[[[ 0.03294463083148002625, -0.13431271910667419434,
           -0.21540719270706176758],
          [ 0.14191532135009765625, -0.14191532135009765625,
           -0.24835182726383209229],
          [ 0.32184368371963500977, -0.14444953203201293945,
           -0.21287299692630767822]]]], size=(1, 1, 3, 3), dtype=torch.qint8,
       quantization_scheme=torch.per_channel_affine,
       scale=tensor([0.00253420230001211166], dtype=torch.float64),
       zero_point=tensor([0]), axis=0)

In [109]:
model_fp32.conv.bias

Parameter containing:
tensor([0.10949999839067459106], requires_grad=True)

### Floating point model

In [2]:
class M(nn.Module):

    def __init__(self):
            super(M, self).__init__()
            # QuantStub converts tensors from floating point to quantized
            # self.BN = nn.BatchNorm2d(3)
            # self.conv = torch.nn.Conv2d(1, 1, 1)
            # self.conv = torch.nn.Conv2d(in_channels=3, out_channels=2, kernel_size=3, stride=1, padding=1)
            self.conv = torch.nn.Conv2d(in_channels=1, out_channels=1, kernel_size=3, stride=1, padding=1)
            # self.conv.weight = torch.nn.Parameter(torch.tensor([[[[ 0.03307433053851127625, -0.13484150171279907227, -0.21625524759292602539], 
            #                                                         [ 0.14247404038906097412, -0.14247404038906097412, -0.24932956695556640625], 
            #                                                         [ 0.32311078906059265137, -0.14501821994781494141, -0.21371106803417205811]]]]))
            # self.conv.weight = torch.nn.Parameter(torch.tensor([[[[ 1.0, 1.0, -1.0], 
            #                                                         [ -1.0, 1.0, 1.0], 
            #                                                         [ 1.0, 1.0, -1.0]]]]).double())
            self.conv.weight = torch.nn.Parameter(torch.tensor([[[[ -1.0, 0.0, 1.0], 
                                                                    [ -1.0, 0.0, 1.0], 
                                                                    [ -1.0, 0.0, 1.0]]]]).double())
            self.conv.bias = torch.nn.Parameter(torch.tensor([0.0]))
            # self.relu = torch.nn.ReLU()
            # DeQuantStub converts tensors from quantized to floating point

    def forward(self, x):
        x = self.conv(x)
         
        return x

In [3]:
input_fp32 = torch.tensor([[[[ 0.95466703176498413086, -1.36212718486785888672,
            0.75253891944885253906,  1.57104063034057617188],
          [ 0.97250884771347045898, -0.67004448175430297852,
           -0.58047348260879516602,  1.30683445930480957031],
          [-0.13423979282379150391,  0.16391958296298980713,
           -0.71688455343246459961,  0.05846109613776206970],
          [ 1.07569837570190429688, -0.06351475417613983154,
           -0.19469638168811798096, -0.09430617839097976685]]]], requires_grad=False)

In [4]:
model_fp32 = M()
model_fp32.eval()

M(
  (conv): Conv2d(1, 1, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
)

In [ ]:
def convolve_2D(input_image: np.array, kernel: np.array) -> np.array:
    k = kernel.shape[0]
    tgt_size = input_image.shape[0]
    padded_image = np.zeros(shape=(tgt_size+int((k+1)/2), tgt_size+int((k+1)/2)))
    padded_image[1:input_image.shape[0]+1,1:input_image.shape[1]+1] = input_image
    convolved_img = np.zeros(shape=(tgt_size, tgt_size))
    for i in range(tgt_size):
        for j in range(tgt_size):
            mat = padded_image[i:i+k, j:j+k, :]
            convolved_img[i, j] = np.sum(np.multiply(mat, kernel))
    return convolved_img

In [5]:
def convolve(input_image: np.array, kernel: np.array, bias: np.array) -> np.array:    
    ### Add description of what dimension is what
    k = kernel.shape[0]
    number_input_row = input_image.shape[0]
    number_input_col = input_image.shape[1]
    input_depth = input_image.shape[2]   # R, G, B channel
    filter_depth = kernel.shape[2]       # filter depth
    filter_num = kernel.shape[3]         # number of filters
    bias_depth = bias.shape[0]

    if input_depth != filter_depth:
        print("Error: Number of channels in both image and filter depth must match.")
        sys.exit()

    if bias_depth != filter_num:
        print("Error: Bias depth and filter number must match.")
        sys.exit()
    
    if np.isnan(input_image):
        print("Input image has NaN values")
        sys.exit()

    if np.isnan(kernel):
        print("Kernel has NaN values")
        sys.exit()

    padded_image = np.zeros(shape=(number_input_row+int((k+1)/2), number_input_col+int((k+1)/2), input_depth))
    padded_image[((k-1)>>1):number_input_row + ((k-1)>>1), ((k-1)>>1):number_input_col + ((k-1)>>1), :] = input_image
    convolved_img = np.zeros(shape=(number_input_row, number_input_col, filter_num))

    for f in range(filter_num):
        for i in range(number_input_row):
            for j in range(number_input_col):
                mat = padded_image[i:i+k, j:j+k, :]
                convolved_img[i, j, f ] = np.sum(np.multiply(mat, kernel[:,:,:,f]))
        
        convolved_img[:, :, f ] += bias[f]


    if np.isnan(convolved_img):
        print("convolved Image has NaN values")
        sys.exit()

    return convolved_img

In [6]:
# filter = np.zeros((5, 5, 3, 2))

filter = np.zeros((3, 3, 3, 2))
filter[:, :, 0, 0] = np.array([[[-1, 0, 1], 
                                   [-1, 0, 1], 
                                   [-1, 0, 1]]])
filter[:, :, 1, 0] = np.array([[[1,   1,  1], 
                                   [0,   0,  0], 
                                   [-1, -1, -1]]])
filter[:, :, 2, 0] = np.array([[[1,   1,  1], 
                                   [0,  1,  0], 
                                   [-1, -1, -1]]])

filter[:, :, 0, 1] = np.array([[[-1, 0, 1], 
                                   [-1, 0, 1], 
                                   [-1, 0, 1]]])
filter[:, :, 1, 1] = np.array([[[1,   1,  1], 
                                   [0,   0,  0], 
                                   [-1, -1, -1]]])
filter[:, :, 2, 1] = np.array([[[1,   1,  1], 
                                   [0,   1,  0], 
                                   [-1, -1, -1]]])
kernel = filter

In [7]:
kernel.shape

(3, 3, 3, 2)

In [8]:
import skimage
input_image = skimage.data.chelsea()

In [9]:
input_image.shape

(300, 451, 3)

In [131]:
convolved_img.shape

(300, 451, 2)

In [124]:
kernel.shape

(3, 3, 3, 2)

In [127]:
np.sum(np.multiply(mat, kernel[:,:,:,f]))

-66.0

In [50]:
img.shape

(300, 451)

In [48]:
model_fp32.conv.weight.detach().numpy()[0][0].shape

(3, 3)

In [53]:
model_fp32.conv.weight.detach().numpy()

array([[[[-1.,  0.,  1.],
         [-1.,  0.,  1.],
         [-1.,  0.,  1.]]]])

In [72]:
convolved_img = convolve(img, model_fp32.conv.weight.detach().numpy()[0][0])

ValueError: could not broadcast input array from shape (300,451) into shape (301,301)

In [52]:
convolved_img

array([[-0.03334471, -0.03334471, -0.00560314, ...,  0.1419949 ,
         0.07033529, -0.01593922],
       [-0.04118784, -0.03334471, -0.00980745, ...,  0.09355216,
         0.02522588, -0.04928392],
       [-0.04175333, -0.02998863, -0.01401176, ...,  0.05882353,
         0.00561804, -0.04144078],
       ...,
       [ 0.2025298 ,  0.23276353,  0.15765059, ..., -0.04734941,
        -0.16304627, -0.07788784],
       [ 0.21147412,  0.14478471,  0.0173302 , ..., -0.02637255,
        -0.14599098, -0.08239765],
       [ 0.10724353, -0.0165651 , -0.1733902 , ..., -0.00058824,
        -0.11854   , -0.09024078]])

In [59]:
model_fp32(input_fp32.double())

tensor([[[[-2.032171666622161865234375000000, -1.755110442638397216796875000000,
            4.910046756267547607421875000000, -0.172065436840057373046875000000],
          [-1.868252083659172058105468750000, -2.337755203247070312500000000000,
            4.804588269442319869995117187500,  0.544819116592407226562500000000],
          [-0.569639652967453002929687500000, -3.406021848320960998535156250000,
            1.840629030019044876098632812500,  1.492054417729377746582031250000],
          [ 0.100404828786849975585937500000, -1.853039517998695373535156250000,
           -0.136249911040067672729492187500,  0.911580935120582580566406250000]]]],
       dtype=torch.float64, grad_fn=<SlowConv2DBackward0>)

In [56]:
input_fp32[0][0].detach().numpy().shape

(4, 4)

In [64]:
input_fp32

tensor([[[[ 0.954667031764984130859375000000, -1.362127184867858886718750000000,
            0.752538919448852539062500000000,  1.571040630340576171875000000000],
          [ 0.972508847713470458984375000000, -0.670044481754302978515625000000,
           -0.580473482608795166015625000000,  1.306834459304809570312500000000],
          [-0.134239792823791503906250000000,  0.163919582962989807128906250000,
           -0.716884553432464599609375000000,  0.058461096137762069702148437500],
          [ 1.075698375701904296875000000000, -0.063514754176139831542968750000,
           -0.194696381688117980957031250000, -0.094306178390979766845703125000]]]])

In [63]:
input_fp32.detach().numpy()

array([[[[ 0.95466703, -1.3621272 ,  0.7525389 ,  1.5710406 ],
         [ 0.97250885, -0.6700445 , -0.5804735 ,  1.3068345 ],
         [-0.1342398 ,  0.16391958, -0.71688455,  0.0584611 ],
         [ 1.0756984 , -0.06351475, -0.19469638, -0.09430618]]]],
      dtype=float32)

In [93]:
model_fp32(input_fp32.double())

tensor([[[[-2.032171666622161865234375000000, -1.755110442638397216796875000000,
            4.910046756267547607421875000000, -0.172065436840057373046875000000],
          [-1.868252083659172058105468750000, -2.337755203247070312500000000000,
            4.804588269442319869995117187500,  0.544819116592407226562500000000],
          [-0.569639652967453002929687500000, -3.406021848320960998535156250000,
            1.840629030019044876098632812500,  1.492054417729377746582031250000],
          [ 0.100404828786849975585937500000, -1.853039517998695373535156250000,
           -0.136249911040067672729492187500,  0.911580935120582580566406250000]]]],
       dtype=torch.float64, grad_fn=<SlowConv2DBackward0>)

In [146]:
filter = np.zeros((3, 3, 1, 1))
# filter[:, :, 1, 1] = model_fp32.conv.weight.detach().numpy()[0][0]
np.reshape(model_fp32.conv.weight.detach().numpy()[0][0], (1, 1, 3, 3))

array([[[[-1.,  0.,  1.],
         [-1.,  0.,  1.],
         [-1.,  0.,  1.]]]])

In [10]:
input_image.shape

(300, 451, 3)

In [11]:
np.reshape(input_fp32[0][0].detach().numpy(), (4, 4, 1)).shape

(4, 4, 1)

In [12]:
model_fp32.conv.weight.detach().numpy()[0][0].shape

(3, 3)

In [29]:
x = np.array([0])

In [30]:
x[0]

0

In [34]:
ans = convolve(np.reshape(input_fp32[0][0].detach().numpy(), (4, 4, 1)), np.reshape(model_fp32.conv.weight.detach().numpy()[0][0], (3, 3, 1, 1)), np.array([1]))

In [35]:
ans.shape

(4, 4, 1)

In [33]:
ans

array([[[-2.03217167],
        [-1.75511044],
        [ 4.91004676],
        [-0.17206544]],

       [[-1.86825208],
        [-2.3377552 ],
        [ 4.80458827],
        [ 0.54481912]],

       [[-0.56963965],
        [-3.40602185],
        [ 1.84062903],
        [ 1.49205442]],

       [[ 0.10040483],
        [-1.85303952],
        [-0.13624991],
        [ 0.91158094]]])

In [89]:
convolve(input_fp32[0][0].detach().numpy(), model_fp32.conv.weight.detach().numpy()[0][0])

array([[-2.03217167, -1.75511044,  4.91004676, -0.17206544],
       [-1.86825208, -2.3377552 ,  4.80458827,  0.54481912],
       [-0.56963965, -3.40602185,  1.84062903,  1.49205442],
       [ 0.10040483, -1.85303952, -0.13624991,  0.91158094]])

In [17]:
model_fp32(input_fp32.double()).detach().numpy()

array([[[[-2.03217167, -1.75511044,  4.91004676, -0.17206544],
         [-1.86825208, -2.3377552 ,  4.80458827,  0.54481912],
         [-0.56963965, -3.40602185,  1.84062903,  1.49205442],
         [ 0.10040483, -1.85303952, -0.13624991,  0.91158094]]]])

In [95]:
np.sum(np.sum(model_fp32(input_fp32.double()).detach().numpy() - convolve(input_fp32[0][0].detach().numpy(), model_fp32.conv.weight.detach().numpy()[0][0])))

0.0

In [75]:
img2 = input_fp32[0][0].detach().numpy()

In [84]:
img2

array([[ 0.95466703, -1.3621272 ,  0.7525389 ,  1.5710406 ],
       [ 0.97250885, -0.6700445 , -0.5804735 ,  1.3068345 ],
       [-0.1342398 ,  0.16391958, -0.71688455,  0.0584611 ],
       [ 1.0756984 , -0.06351475, -0.19469638, -0.09430618]],
      dtype=float32)

### Code from google

In [5]:
def conv_(img, conv_filter):
    filter_size = conv_filter.shape[1]
    result = np.zeros((img.shape))
    #Looping through the image to apply the convolution operation.
    for r in np.uint16(np.arange(filter_size/2.0, 
                          img.shape[0]-filter_size/2.0+1)):
        for c in np.uint16(np.arange(filter_size/2.0, 
                                           img.shape[1]-filter_size/2.0+1)):
            """
            Getting the current region to get multiplied with the filter.
            How to loop through the image and get the region based on 
            the image and filer sizes is the most tricky part of convolution.
            """
            curr_region = img[r-np.uint16(np.floor(filter_size/2.0)):r+np.uint16(np.ceil(filter_size/2.0)), 
                              c-np.uint16(np.floor(filter_size/2.0)):c+np.uint16(np.ceil(filter_size/2.0))]
            #Element-wise multipliplication between the current region and the filter.
            curr_result = curr_region * conv_filter
            conv_sum = np.sum(curr_result) #Summing the result of multiplication.
            result[r, c] = conv_sum #Saving the summation in the convolution layer feature map.
            
    #Clipping the outliers of the result matrix.
    final_result = result[np.uint16(filter_size/2.0):result.shape[0]-np.uint16(filter_size/2.0), 
                          np.uint16(filter_size/2.0):result.shape[1]-np.uint16(filter_size/2.0)]
    return final_result

In [6]:
def conv(img, conv_filter):
    if len(img.shape) > 2 or len(conv_filter.shape) > 3: # Check if number of image channels matches the filter depth.
        if img.shape[-1] != conv_filter.shape[-1]:
            print("Error: Number of channels in both image and filter must match.")
            sys.exit()
    if conv_filter.shape[1] != conv_filter.shape[2]: # Check if filter dimensions are equal.
        print('Error: Filter must be a square matrix. I.e. number of rows and columns must match.')
        sys.exit()
    if conv_filter.shape[1]%2==0: # Check if filter diemnsions are odd.
        print('Error: Filter must have an odd size. I.e. number of rows and columns must be odd.')
        sys.exit()

    # An empty feature map to hold the output of convolving the filter(s) with the image.
    feature_maps = np.zeros((img.shape[0]-conv_filter.shape[1]+1, 
                                img.shape[1]-conv_filter.shape[1]+1, 
                                conv_filter.shape[0]))

    # Convolving the image by the filter(s).
    for filter_num in range(conv_filter.shape[0]):
        print("Filter ", filter_num + 1)
        curr_filter = conv_filter[filter_num, :] # getting a filter from the bank.
        """ 
        Checking if there are mutliple channels for the single filter.
        If so, then each channel will convolve the image.
        The result of all convolutions are summed to return a single feature map.
        """
        if len(curr_filter.shape) > 2:
            conv_map = conv_(img[:, :, 0], curr_filter[:, :, 0]) # Array holding the sum of all feature maps.
            for ch_num in range(1, curr_filter.shape[-1]): # Convolving each channel with the image and summing the results.
                conv_map = conv_map + conv_(img[:, :, ch_num], 
                                  curr_filter[:, :, ch_num])
        else: # There is just a single channel in the filter.
            conv_map = conv_(img, curr_filter)
        feature_maps[:, :, filter_num] = conv_map # Holding feature map with the current filter.
    return feature_maps # Returning all feature maps.

In [14]:
import skimage
# Reading the image
#img = skimage.io.imread("fruits2.png")
img = skimage.data.chelsea()
# Converting the image into gray.
img = skimage.color.rgb2gray(img)

In [15]:

# First conv layer
#l1_filter = numpy.random.rand(2,7,7)*20 # Preparing the filters randomly.
l1_filter = np.zeros((1,3,3))
l1_filter[0, :, :] = np.array([[[-1, 0, 1], 
                                   [-1, 0, 1], 
                                   [-1, 0, 1]]])
# l1_filter[1, :, :] = np.array([[[1,   1,  1], 
#                                    [0,   0,  0], 
#                                    [-1, -1, -1]]])
print("\n**Working with conv layer 1**")
l1_feature_map = conv(img, l1_filter)
print("\n**ReLU**")


**Working with conv layer 1**
Filter  1

**ReLU**


In [13]:
input_fp32[0].detach().numpy().shape[-1]

4

In [16]:
l1_feature_map = conv(img, l1_filter)

Filter  1


In [18]:
l1_feature_map.shape

(298, 449, 1)

In [29]:
img_tensor = torch.tensor(img).double()

In [30]:
img_tensor.resize_(1, 1, img.shape[0], img.shape[1])

tensor([[[[0.485230980392156840963480135542, 0.485230980392156840963480135542,
           0.477387843137254908043587420252,  ...,
           0.116923921568627436906595562505, 0.116923921568627436906595562505,
           0.116923921568627436906595562505],
          [0.496995686274509795854470439735, 0.493074117647058829394524082090,
           0.485230980392156840963480135542,  ...,
           0.123368235294117639666566788037, 0.122534901960784300967155502349,
           0.127289803921568633882088761311],
          [0.508492549019607831262135277939, 0.504570980392156864802188920294,
           0.494755686274509776101382385605,  ...,
           0.123360392156862744283962740610, 0.127281960784313724621696906070,
           0.131203529411764691081643263715],
          ...,
          [0.247867450980392145520880831100, 0.298847843137254876033637174260,
           0.405012941176470575577184263238,  ...,
           0.589148235294117639249122930778, 0.589148235294117639249122930778,
           

In [31]:
img_tensor.shape

torch.Size([1, 1, 300, 451])

In [43]:
model_fp32(img_tensor)

tensor([[[[ 9.783050980392156148468529863749e-01,
           -1.960784313725483229973178822547e-02,
           -1.960784313725483229973178822547e-02,
            ...,
           -1.060313725490198699663579873231e-02,
            3.921568627450994215521973274008e-03,
           -2.394588235294117239959632570390e-01],
          [ 1.482876078431372590671344369184e+00,
           -3.334470588235288746048468055960e-02,
           -3.334470588235288746048468055960e-02,
            ...,
           -8.921568627450998656414071774634e-03,
            1.176470588235294101320249637865e-02,
           -3.667407843137254763732357787376e-01],
          [ 1.510341960784313819488033914240e+00,
           -4.118784313725498691383108962327e-02,
           -3.334470588235305399393837433308e-02,
            ...,
            3.103137254901966457509843166918e-03,
            1.625176470588236832259809716561e-02,
           -3.874796078431372303718660532468e-01],
          ...,
          [ 9.25683921568627443

In [44]:
l1_feature_map

array([[[-3.33447059e-02],
        [-3.33447059e-02],
        [-5.60313725e-03],
        ...,
        [-1.87141176e-02],
        [-8.92156863e-03],
        [ 1.17647059e-02]],

       [[-4.11878431e-02],
        [-3.33447059e-02],
        [-9.80745098e-03],
        ...,
        [-1.45325490e-02],
        [ 3.10313725e-03],
        [ 1.62517647e-02]],

       [[-4.17533333e-02],
        [-2.99886275e-02],
        [-1.40117647e-02],
        ...,
        [-1.42725490e-02],
        [ 5.33529412e-03],
        [ 1.62517647e-02]],

       ...,

       [[ 2.02529804e-01],
        [ 2.32763529e-01],
        [ 1.57650588e-01],
        ...,
        [-5.86313725e-03],
        [ 1.98000000e-03],
        [-3.92156863e-03]],

       [[ 2.11474118e-01],
        [ 1.44784706e-01],
        [ 1.73301961e-02],
        ...,
        [ 1.10901961e-03],
        [ 5.03058824e-03],
        [-1.11022302e-16]],

       [[ 1.07243529e-01],
        [-1.65650980e-02],
        [-1.73390196e-01],
        ...,
        